In [1]:
from pipeline.pipeline_1 import run_pipeline_1

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    result_path = "result/p1_matching_matrix.npy"
    embedding_cache_path = "embedding/Pipeline_1_embeddings_cache.pkl"

    # Run Pipeline 1
    run_pipeline_1(file_path_excel, file_paths_csv, embedding_cache_path, result_path)


if __name__ == "__main__":
    main()


/home/fcr/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-20 11:08:23.318739: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 11:08:23.325610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734689303.334165  328061 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734689303.336814  328061 cuda_blas.cc:1418] Unable to reg

Loading embeddings from cache...
Length of guideline_embeddings   : 79
Length of requirement_embeddings : 734
Performing semantic search...


Batches: 100%|██████████| 3/3 [00:00<00:00,  8.92it/s]

Semantic search completed in 0.39 seconds.
Matrix saved to result/p1_matching_matrix.npy


In [ ]:
from pipeline.pipeline_2 import run_pipeline_2

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = [
        "DATA/1_Harmonisiert_pflichtenheft.csv",
        "DATA/2_Harmonisiert_pflichtenheft.csv",
        "DATA/4_Harmonisiert_ausschreibung.csv"
    ]
    embedding_cache_path = "embedding/Pipeline_2_embeddings_cache.pkl"
    result_path = "result/Pipeline_2.csv"

    # Set hyperparameters
    top_k = 10  # Number of top reranked matches to retrieve
    bi_top_k = 32  # Number of top Bi-Encoder matches before reranking

    # Run Pipeline 2
    run_pipeline_2(file_path_excel, file_paths_csv, embedding_cache_path, result_path, top_k=top_k, bi_top_k=bi_top_k)


if __name__ == "__main__":
    main()

In [1]:
from pipeline.baseline import run_baseline

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    result_path = "result/p1_matching_matrix.npy"

    # Set hyperparameters
    model_name = "gpt-4o-mini"  # Specify GPT model

    # Run Baseline Pipeline
    run_baseline(file_path_excel, file_paths_csv, result_path, model_name=model_name)


if __name__ == "__main__":
    main()

Performing GPT-based semantic search...
Processing sentence: Sie verwenden Öle, Gewürze, Streusalze, Zucker, Ba...
no match: {'text': 'The sentence discusses the use of organic oils, spices, salts, sugars, baked goods, and other basic products and foods, with a focus on 50% of these being of "Bio" quality and products from the Global South having the "Max Havelaar" quality seal. Based on this context, the most relevant guidelines are:'}
no match: {'text': '1. **Mindestens 10% des Gesamtgewichts aller Lebensmittel stammen aus dem Biolandbau und haben ein Biolabel, das in offiziellen Leitfäden empfohlen wird.** (At least 10% of the total weight of all food comes from organic farming and has a bio label recommended in official guidelines.)'}
no match: {'text': '2. **Mindestens X% des Gesamtgewichts der exotischen Produkte (1) (insbesondere Kaffee, Kakao, Schokolade, exotische Früchte und Fruchtsäfte) haben ein Label, das in offiziellen Leitfäden empfohlen wird.** (At least X% of the total

KeyboardInterrupt: 

In [ ]:
from pipeline.pipeline_3 import run_pipeline_3
import nest_asyncio

# 允许嵌套事件循环
nest_asyncio.apply()

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    result_path = "result/p3_matching_matrix.npy"
    embedding_cache_path = "embedding/Pipeline_2_embeddings_cache.pkl"

    # Set hyperparameters
    model_name = "gpt-4o-mini"  # Specify GPT model

    # Run Baseline Pipeline
    run_pipeline_3(file_path_excel, file_paths_csv, embedding_cache_path, result_path, gpt_model_name=model_name)


if __name__ == "__main__":
    main()

In [7]:
from pipeline.evaluate import recall_k, unrelated_acc
import numpy as np

target_matrix_path = 'matched_table.csv'
target_matrix = np.loadtxt(target_matrix_path, delimiter=",", dtype=int)

predicted_matrix_path = "result/p3_matching_matrix.npy"
predicted_matrix = np.load(predicted_matrix_path)

unrelated_score = unrelated_acc(predicted_matrix)
recall_score = recall_k(predicted_matrix[:30], target_matrix[:30])

print(f"the unrelated acc of pipeline 3 is: {unrelated_score}")
print(f"the recall k score of pipeline 3 is: {recall_score}")

the unrelated acc of pipeline 3 is: 0.7176470588235294
the recall k score of pipeline 3 is: 0.3902777777777777


In [8]:
target_matrix_path = 'matched_table.csv'
target_matrix = np.loadtxt(target_matrix_path, delimiter=",", dtype=int)

predicted_matrix_path = "result/p4_matching_matrix.npy"
predicted_matrix = np.load(predicted_matrix_path)

unrelated_score = unrelated_acc(predicted_matrix)
recall_score = recall_k(predicted_matrix[:30], target_matrix[:30])

print(f"the unrelated acc of pipeline 3 is: {unrelated_score}")
print(f"the recall k score of pipeline 3 is: {recall_score}")

the unrelated acc of pipeline 3 is: 0.6588235294117647
the recall k score of pipeline 3 is: 0.43749999999999994


In [9]:
target_matrix_path = 'matched_table.csv'
target_matrix = np.loadtxt(target_matrix_path, delimiter=",", dtype=int)

predicted_matrix_path = "result/p5_matching_matrix.npy"
predicted_matrix = np.load(predicted_matrix_path)

unrelated_score = unrelated_acc(predicted_matrix)
recall_score = recall_k(predicted_matrix[:30], target_matrix[:30])

print(f"the unrelated acc of pipeline 3 is: {unrelated_score}")
print(f"the recall k score of pipeline 3 is: {recall_score}")

the unrelated acc of pipeline 3 is: 0.7058823529411765
the recall k score of pipeline 3 is: 0.5777777777777777


In [ ]:
target_matrix_path = 'matched_table.csv'
target_matrix = np.loadtxt(target_matrix_path, delimiter=",", dtype=int)

predicted_matrix_path = "result/p1_matching_matrix.npy"
predicted_matrix = np.load(predicted_matrix_path)

unrelated_score = unrelated_acc(predicted_matrix)
recall_score = recall_k(predicted_matrix[:30], target_matrix[:30])

print(f"the unrelated acc of pipeline 3 is: {unrelated_score}")
print(f"the recall k score of pipeline 3 is: {recall_score}")